In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from sklearn.utils import class_weight

In [9]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#load data from drive data/00_Data.xlsx
df = pd.read_excel('/content/drive/MyDrive/data/00_Data.xlsx')

#print NA count
print(df.isna().sum())

# Collect ground truth or labels
y = df['DEFECTIVE']

# Collect all features
X = df.drop('DEFECTIVE', axis=1)

#Encode labels in class 0 and 1
le = LabelEncoder()
y = le.fit_transform(y)



PARAMETER 1     0
PARAMETER 2     0
PARAMETER 3     0
PARAMETER 7     0
PARAMETER 8     0
PARAMETER 10    0
PARAMETER 11    0
PARAMETER 12    0
PARAMETER 13    0
PARAMETER 14    0
PARAMETER 15    0
PARAMETER 16    0
PARAMETER 17    0
PARAMETER 18    0
PARAMETER 19    0
PARAMETER 20    0
PARAMETER 21    0
PARAMETER 22    0
PARAMETER 23    0
PARAMETER 24    0
PARAMETER 25    0
DEFECTIVE       0
dtype: int64


In [11]:
## print classes
print(le.classes_)  # Output: ['NO', 'YES']
print(dict(zip(le.classes_, le.transform(le.classes_))))

['DEFECTIVE' 'NO']
{'DEFECTIVE': np.int64(0), 'NO': np.int64(1)}


In [12]:
#Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns=df.columns[:-1])

In [13]:
#test train split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# Compute class weights
class_weights_array = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights_array))
print("Class Weights:", class_weight_dict)


Class Weights: {0: np.float64(3.5215123859191655), 1: np.float64(0.5827400215749731)}


In [ ]:

# Build ANN model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile and train
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, validation_split=0.2, batch_size=32,class_weight=class_weight_dict )



Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


406/406 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6072 - loss: 0.6358 - val_accuracy: 0.7239 - val_loss: 0.5362
Epoch 2/100
406/406 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7332 - loss: 0.5290 - val_accuracy: 0.7051 - val_loss: 0.5548
Epoch 3/100
406/406 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7361 - loss: 0.5164 - val_accuracy: 0.7406 - val_loss: 0.5245
Epoch 4/100
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7503 - loss: 0.5109 - val_accuracy: 0.7906 - val_loss: 0.4617
Epoch 5/100
406/406 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7685 - loss: 0.4939 - val_accuracy: 0.7776 - val_loss: 0.4648
Epoch 6/100
406/406 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7655 - loss: 0.4972 - val_accuracy: 0.7862 - val_loss: 0.4631
Epoch 7/100
406/406 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7949 - loss: 0.4678 - val_accuracy: 0.7736 - val_loss: 0.4871
Epoch 8/100
406/406 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7792 - loss: 0.4716 - val_accuracy: 0.7610

In [19]:
# Evaluate
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
              precision    recall  f1-score   support

           0       0.42      0.80      0.55       586
           1       0.96      0.81      0.88      3466

    accuracy                           0.81      4052
   macro avg       0.69      0.81      0.71      4052
weighted avg       0.88      0.81      0.83      4052



This was obtained without class weightage.    
0 => DEFECTIVE
1 => NO    
              precision    recall  f1-score   support

           0       0.69      0.38      0.49       586
           1       0.90      0.97      0.94      3466

    accuracy                           0.89      4052
   macro avg       0.80      0.68      0.71      4052
weighted avg       0.87      0.89      0.87      4052